### 3. 모델링 기초(2)
- 위도, 경도 시각화
- 범주형 데이터 변환

### 1. 데이터 읽어오기

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

### 2. 데이터 확인

In [ ]:
train[['next_latitude',	'next_longitude']][:5].values.tolist()

In [ ]:
import folium

# 좌표 정보
locations = train[['next_latitude', 'next_longitude']][:5].values.tolist()

# 지도의 중심 좌표 설정
center = [33.48, 126.53]

# 지도 (Map) 그리기
m = folium.Map(location=center, zoom_start=13, tiles='cartodbpositron',)

# 점(Circle) 찍기
for i, location in enumerate(locations):
    folium.Circle(
        radius=50,
        location=location,
        tooltip=train['next_station'].loc[i],
        fill=True
    ).add_to(m)

# 경로(PolyLIne) 추가
folium.PolyLine(locations=locations).add_to(m)

# 결과 출력
m

### 3. 데이터 전처리

In [ ]:
train['now_station'].nunique(), train['next_station'].nunique()

In [ ]:
set(train['next_station'].unique()) - set(train['now_station'].unique())

In [ ]:
my_list = list(train['route_nm'].unique()) + list(train['now_station'].unique()) + list(train['next_station'].unique())
my_dict = {text : i for i, text in enumerate(my_list)}
my_dict

In [ ]:
train[['route_nm', 'now_station', 'next_station']] = train[['route_nm', 'now_station', 'next_station']].applymap(lambda x : my_dict[x])
train.head()

In [ ]:
# 시간 데이터 타입 변환 함수
def text_to_time(text:str):
    return int(text[:2])

train['now_arrive_time'] = train['now_arrive_time'].map(text_to_time)
train.head()

In [ ]:
# my_dict 딕셔너리를 사용하여 각 칼럼의 값을 정수로 변환하는 코드
test[['route_nm', 'now_station', 'next_station']] = test[['route_nm', 'now_station', 'next_station']].applymap(lambda x : my_dict[x])

# 시간 데이터 타입 변환 함수
def text_to_time(text:str):
    return int(text[:2])

# 'xx시' 와 같은 값을 정수로 변환하는 코드
test['now_arrive_time'] = test['now_arrive_time'].map(text_to_time)

In [ ]:
y = train['next_arrive_time']

y.head()

In [ ]:
features = train.columns[2:-1]     
X = train[features]
X.head()

In [ ]:
drop_list = ['id','date','route_nm','now_station','now_arrive_time','next_station']
train = train.drop(drop_list, axis = 1)

In [ ]:
drop_list = ['id','date','route_nm','now_station','now_arrive_time','next_station']
test = test.drop(drop_list, axis = 1)

In [ ]:
# 예측할 변수 y 할당
y = train['next_arrive_time']

# y 데이터를 다시 살펴봅니다.
y.head()

In [ ]:
# 학습에 사용할 변수 X 할당
X = train[['route_id', 'vh_id', 'now_latitude', 'now_longitude', 'distance', 'next_latitude', 'next_longitude']]

### 4. 데이터 학습

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [ ]:
# 모델 학습
model.fit(X, y)

In [ ]:
# 학습된 모델을 이용해 결괏값 예측 후 상위 10개의 값 확인
predict = model.predict(test)
print('----------------------예측된 데이터의 상위 10개의 값 확인--------------------\n')
print(predict[:10])

### 5. CSV파일로 저장

In [ ]:
submission['next_arrive_time'] = predict
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)